In [22]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import random

In [23]:
df_ebay = pd.read_csv('../data/processed/ebay_expanded.csv')

In [24]:
df_agg = pd.read_csv("hf://datasets/lakritidis/product-matching/pricerunner_aggregate.csv")

In [25]:
df_agg

,Product ID,Product Title,Vendor ID,Cluster ID,Cluster Label,Category ID,Category Label
0,1,apple iphone 8 plus 64gb silver,1,1,Apple iPhone 8 Plus 64GB,2612,Mobile Phones
1,2,apple iphone 8 plus 64 gb spacegrau,2,1,Apple iPhone 8 Plus 64GB,2612,Mobile Phones
2,3,apple mq8n2b/a iphone 8 plus 64gb 5.5 12mp sim...,3,1,Apple iPhone 8 Plus 64GB,2612,Mobile Phones
3,4,apple iphone 8 plus 64gb space grey,4,1,Apple iPhone 8 Plus 64GB,2612,Mobile Phones
4,5,apple iphone 8 plus gold 5.5 64gb 4g unlocked ...,5,1,Apple iPhone 8 Plus 64GB,2612,Mobile Phones
...,...,...,...,...,...,...,...
35306,47350,smeg fab28 60cm retro style right hand hinge f...,59,47517,Smeg FAB28 Cream,2623,Fridges
35307,47351,smeg fab28 60cm retro style left hand hinge fr...,59,47518,Smeg FAB28 Red,2623,Fridges
35308,47352,smeg fab28 60cm retro style left hand hinge fr...,59,47519,Smeg FAB28 Pink,2623,Fridges
35309,47355,candy 60cm built under larder fridge cru160nek,125,47524,Candy CRU16.0,2623,Fridges


In [26]:
cluster_dict = df_agg.groupby('Cluster Label').apply(lambda x: x['Product Title'].values).to_dict()
cluster_dict['Apple iPhone 8 Plus 64GB']

/tmp/ipykernel_4754/2237328586.py:1: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  cluster_dict = df_agg.groupby('Cluster Label').apply(lambda x: x['Product Title'].values).to_dict()


array(['apple iphone 8 plus 64gb silver',
       'apple iphone 8 plus 64 gb spacegrau',
       'apple mq8n2b/a iphone 8 plus 64gb 5.5 12mp sim free smartphone in gold',
       'apple iphone 8 plus 64gb space grey',
       'apple iphone 8 plus gold 5.5 64gb 4g unlocked sim free',
       'apple iphone 8 plus gold 5.5 64gb 4g unlocked sim free',
       'apple iphone 8 plus 64 gb space grey',
       'apple iphone 8 plus 64gb space grey',
       'apple iphone 8 plus 64gb space grey',
       'apple iphone 8 plus 64gb space grey',
       'apple iphone 8 plus 5.5 single sim 4g 64gb silver',
       'sim free iphone 8 plus 64gb by apple space grey',
       'apple iphone 8 plus 64gb gold smartphone',
       'apple iphone 8 plus 5.5 single sim 4g 64gb grey',
       'apple iphone 8 plus silver 5.5 64gb 4g unlocked sim free',
       'apple iphone 8 plus 64 gb silver',
       'apple iphone 8 plus 64gb silver unlocked',
       'apple iphone 8 plus 14 cm 5.5 64 gb 12 mp ios 11 silver',
       'iphone 8

In [27]:
corr_clust = {}
corr_clust_set = set()

for key in tqdm(cluster_dict.keys()):
    if key in corr_clust_set:
        continue
    corr_clust_set.add(key)
    for pair in cluster_dict.keys():
        if key != pair and  key.split()[0].lower() == pair.split()[0].lower():
            corr_clust[key] = corr_clust.get(key, [key]) + [pair]
            corr_clust_set.add(pair)

100%|██████████| 12849/12849 [00:00<00:00, 12899.84it/s]


In [ ]:
data = []
dict_keys = list(cluster_dict.keys())

# create close pairs
# each pair build from common cluster
for key in cluster_dict.keys():
    if len(cluster_dict[key]) == 1:
        data.append({'query' : key,
                     'word' : cluster_dict[key][0],
                     'label' : 1})
    else:
        sim_word1, sim_word2 = random.sample(list(cluster_dict[key]), 2)
        data.append({'query' : sim_word1,
                    'word' : sim_word2,
                    'label' : 1})
        sim_word1, sim_word2 = random.sample(list(cluster_dict[key]), 2)
        data.append({'query' : key,
                    'word' : sim_word1,
                    'label' : 1})
        data.append({'query' : key,
                    'word' : sim_word2,
                    'label' : 1})
        # sim_word = random.choice(cluster_dict[key])
        # if sim_word != word:
        #     data.append({'query' : word,
        #                  'word' : sim_word,
        #                  'label' : 1})
            
        # sim_word = random.choice(cluster_dict[key])
        # if sim_word != word:
        #     data.append({'query' : word,
        #                  'word' : sim_word,
        #                  'label' : 1})

# create different pairs
for key in corr_clust.keys():
    for ind in range(0, len(corr_clust[key]) - 1, 2):
        data.append({'query' : corr_clust[key][ind],
                     'word' : corr_clust[key][ind + 1],
                     'label' : 0})
        word1_1, word1_2 = random.sample(cluster_dict[corr_clust[key][ind]], 2)
        word2_1, word2_2 = random.sample(cluster_dict[corr_clust[key][ind + 1]], 2)
        
        data.append({'query' : word1_1,
                     'word' : word2_1,
                     'label' : 0})
        
        data.append({'query' : word1_2,
                     'word' : word2_2,
                     'label' : 0})
        

In [48]:
processed_df = pd.DataFrame(data[::-1])

In [49]:
processed_df

,query,word,label
0,iceq 95 litre slimline under counter refrigera...,iceq 48l table top mini drinks beer fridge loc...,0
1,iceQ IceQ95w White,iceQ Iceq48b Black,0
2,iceq 93 litre under counter glass door display...,iceq 96 litre fridge freezer,0
3,iceQ ICEQ93G Black,iceQ ICEQ96FFW White,0
4,iceq 70 litre compact counter top table mini d...,iceq 70 litre table top fridge white,0
...,...,...,...
41202,AEG A72700GNX0 Silver,aeg a72700gnx0 60cm freestanding frost free fr...,1
41203,AEG A72020GNW0 White,aeg a72020gnw0/a72020gnw0,1
41204,AEG A72020GNW0 White,aeg a72020gnw0 freezer 60cm a energy rating in...,1
41205,aeg a72020gnw0 freezer 60cm a energy rating in...,aeg a72020gnw0/a72020gnw0,1


In [50]:
processed_df = pd.concat([df_ebay, processed_df], ignore_index=True)

In [51]:
processed_df.groupby('label').count()

,Unnamed: 0,query,word
label,,,
0.0,4248,16926,16926
1.0,4298,32827,32827


In [40]:
processed_df

,query,word,label
0,Hisense U8N ULED 75 4K Smart TV,"Hisense U8N 75"" ULED TV",0.0
1,Hisense U8N ULED 75 4K Smart TV,"Hisense U8N 75"" ULED TV",0.0
2,Ayaneo KUN 64GB RAM 4TB SSD,Alienware m18 R2 Intel i9-14900HX 64GB RAM 4TB...,0.0
3,Ayaneo KUN 64GB RAM 4TB SSD,Alienware m18 R2 Intel i9-14900HX 64GB RAM 4TB...,0.0
4,Ayaneo KUN 64GB RAM 4TB SSD,OneXPlayer 2 Pro 64GB RAM 4TB SSD,0.0
...,...,...,...
43409,AEG A72700GNX0 Silver,aeg a72700gnx0 60cm freestanding frost free fr...,1.0
43410,AEG A72020GNW0 White,aeg a72020gnw0 freezer 60cm a energy rating in...,1.0
43411,AEG A72020GNW0 White,aeg a72020gnw0/a72020gnw0,1.0
43412,aeg a72020gnw0 freezer 60cm a energy rating in...,aeg a72020gnw0/a72020gnw0,1.0


In [39]:
processed_df.drop('Unnamed: 0', axis=1, inplace=True)

In [33]:
processed_df.to_csv('../data/processed/combined_data.csv')